# Kullanıcı Bazlı VectorStore Sınıfı Dokümantasyonu

**Oluşturan:** [Emre Yağız Koçak]
**Tarih:** 13.10.2025

### Amaç
Bu notebook, her kullanıcı için ayrı bir koleksiyon yönetimi sağlayan `VectorStore` sınıfını belgelemektedir. Sınıf, bir `user_id` ile başlatılır ve tüm veritabanı işlemlerini (ekleme, arama, silme) bu kullanıcıya özel koleksiyon üzerinde gerçekleştirir. Bu yapı, çok kullanıcılı bir sistemde veri izolasyonunu sağlar.

In [1]:
import chromadb
from typing import Optional, List
from langchain_core.documents import Document

class VectorStore:
    def __init__(self, user_id: str):
        """
        Sınıfı belirtilen kullanıcı ID'si için başlatır.
        Kalıcı bir veritabanı istemcisi oluşturur ve kullanıcıya özel bir koleksiyon yükler.
        """
        self.user_id = user_id
        # Veritabanını kalıcı olarak 'chroma_db' klasöründe saklar
        self.client = chromadb.PersistentClient(path="./chroma_db")
        # Her kullanıcı için, ID'sini içeren benzersiz bir koleksiyon adı oluşturur
        self.collection = self.client.get_or_create_collection(
            name=f'user_{user_id}_collection'
        )
        print(f"Kullanıcı '{user_id}' için VectorStore başlatıldı. Koleksiyon: '{self.collection.name}'")

    def add_user_documents(self, documents: List[Document]):
        """Kullanıcının yüklediği dökümanları koleksiyona ekler."""
        # Yeni doküman eklemeden önce mevcut koleksiyonu temizleyelim.
        if self.collection.count() > 0:
            print("Mevcut koleksiyon temizleniyor...")
            self.delete_collection()
            self.collection = self.client.get_or_create_collection(name=f'user_{self.user_id}_collection')

        print(f"{len(documents)} adet doküman koleksiyona ekleniyor...")
        contents = [doc.page_content for doc in documents]
        metadatas = [{"user_id": self.user_id, **doc.metadata} for doc in documents]
        ids = [f"user_{self.user_id}_doc_{i}" for i in range(len(documents))]
        self.collection.add(documents=contents, metadatas=metadatas, ids=ids)
        print("Dokümanlar başarıyla eklendi.")

    def find_document(self, query: str) -> Optional[Document]:
        """Sadece bu kullanıcının koleksiyonunda en alakalı tek dokümanı arar."""
        results = self.collection.query(
            query_texts=[query],
            n_results=1,
            include=["metadatas", "documents"]
        )
        if results and results['documents'][0]:
            page_content = results['documents'][0][0]
            metadata = results['metadatas'][0][0]
            return Document(page_content=page_content, metadata=metadata)
        return None

    def delete_collection(self):
        """Mevcut kullanıcının koleksiyonunu veritabanından tamamen siler."""
        print(f"\n'{self.collection.name}' koleksiyonu siliniyor...")
        try:
            self.client.delete_collection(name=self.collection.name)
            print("Koleksiyon başarıyla silindi.")
        except Exception as e:
            print(f"Koleksiyon silinirken bir hata oluştu: {e}")

print("VectorStore sınıfı başarıyla tanımlandı ve kullanıma hazır.")

VectorStore sınıfı başarıyla tanımlandı ve kullanıma hazır.


### Canlı Test ve Kullanım Örneği

Aşağıdaki hücreler, `VectorStore` sınıfının nasıl kullanılacağını adım adım göstermektedir.

In [2]:
# Test için bir kullanıcı ID'si belirleyelim
test_user_id = "user_abc_123"

# VectorStore'u bu kullanıcı için başlatalım
# Bu işlem, 'chroma_db' klasörü altında bu kullanıcıya özel bir koleksiyon oluşturacak veya yükleyecektir.
user_vector_store = VectorStore(user_id=test_user_id)

Kullanıcı 'user_abc_123' için VectorStore başlatıldı. Koleksiyon: 'user_user_abc_123_collection'


In [3]:
# Kullanıcının ekleyeceği sahte dokümanları oluşturalım
sample_docs = [
    Document(page_content="Yapay zeka etiği, algoritmaların adil ve şeffaf olmasını amaçlar.", metadata={"source": "makale_1.pdf"}),
    Document(page_content="Ankara, Türkiye'nin başkenti ve en kalabalık ikinci şehridir.", metadata={"source": "cografya_kitabi.pdf"}),
    Document(page_content="Fotosentez, bitkilerin güneş ışığını kullanarak besin üretme sürecidir.", metadata={"source": "biyoloji_ders_notu.pdf"})
]

# Bu dokümanları kullanıcının koleksiyonuna ekleyelim
user_vector_store.add_user_documents(documents=sample_docs)

# Koleksiyonun güncel durumunu kontrol edelim
print(f"Ekleme sonrası koleksiyondaki doküman sayısı: {user_vector_store.collection.count()}")

3 adet doküman koleksiyona ekleniyor...
Dokümanlar başarıyla eklendi.
Ekleme sonrası koleksiyondaki doküman sayısı: 3


In [4]:
# Bir arama yaparak `find_document` metodunu test edelim
search_query = "Ülkemizin başkenti neresidir?"
found_doc = user_vector_store.find_document(search_query)

print(f"'{search_query}' araması için bulunan sonuç:")
if found_doc:
    print(f"  İçerik: {found_doc.page_content}")
    print(f"  Metadata: {found_doc.metadata}")
else:
    print("  İlgili doküman bulunamadı.")

'Ülkemizin başkenti neresidir?' araması için bulunan sonuç:
  İçerik: Ankara, Türkiye'nin başkenti ve en kalabalık ikinci şehridir.
  Metadata: {'source': 'cografya_kitabi.pdf', 'user_id': 'user_abc_123'}


In [5]:
# Test bittikten sonra temizlik için `delete_collection` metodunu çağıralım
# Bu, kullanıcının tüm verilerini veritabanından kaldıracaktır.
user_vector_store.delete_collection()

# Silme sonrası koleksiyonun durumunu kontrol etmeye çalışalım (hata vermesi beklenir)
try:
    # Yeniden bir istemci oluşturup koleksiyonu sorgulamaya çalışalım
    client = chromadb.PersistentClient(path="./chroma_db")
    collection = client.get_collection(name=f'user_{test_user_id}_collection')
    print(f"Silme sonrası doküman sayısı: {collection.count()}")
except Exception as e:
    print(f"\nSilme sonrası koleksiyona erişmeye çalışırken beklenen hata alındı, işlem başarılı: {e}")


'user_user_abc_123_collection' koleksiyonu siliniyor...
Koleksiyon başarıyla silindi.

Silme sonrası koleksiyona erişmeye çalışırken beklenen hata alındı, işlem başarılı: Collection [user_user_abc_123_collection] does not exists


### Değerlendirme: VectorStore Modülünün Projedeki Önemi ve İşlevi

Bu notebook'ta geliştirilen `VectorStore` sınıfı, "PaperMate" projesinin sadece bir parçası değil, aynı zamanda en temel yapı taşlarından biridir. Bu modülün projedeki rolünü ve önemini şu şekilde özetleyebiliriz:

**1. Projenin Uzun Süreli Hafızası:**
VectorStore, işlenen tüm akademik makalelerin anlamsal olarak depolandığı "uzun süreli hafızadır". Team 1'in PDF'lerden çıkardığı ham metinler, bu sistem sayesinde anlamsal bir zekaya sahip, aranabilir bir bilgi kütüphanesine dönüşür. Bu hafıza olmadan, her kullanıcı sorusunda tüm PDF'leri baştan sona taramak zorunda kalırdık, ki bu hem çok yavaş hem de verimsiz olurdu.

**2. Anlamsal Arama (Keyword Aramanın Ötesi):**
Geleneksel veritabanları anahtar kelime (keyword) ile arama yaparken, VectorStore **anlam** ile arama yapar. Bir kullanıcı "mitoz bölünmenin aşamaları" diye sorduğunda, metinde tam olarak bu ifade geçmese bile "hücre bölünmesinin evreleri" gibi anlamsal olarak yakın olan kısımları bulup getirebilir. [cite_start]Bu yetenek, projenin "akıllı" asistan olmasını sağlayan temel özelliktir. [cite: 4]

**3. RAG Sisteminin "R" Harfi (Retrieval):**
Projemizin kalbinde yatan RAG (Retrieval-Augmented Generation) modelinin ilk ve en önemli adımı "Retrieval" (Bilgiyi Getirme)'dir. `VectorStore` sınıfının `similarity_search` (bizim kodumuzdaki `find_document`) metodu, bu adımın doğrudan karşılığıdır. Team 2'nin geliştirdiği dil modeli (LLM), ne kadar zeki olursa olsun, doğru bilgiye erişemediği sürece doğru cevaplar üretemez. Bizim modülümüz, LLM'e cevap üretmesi için gereken doğru ve alakalı "bağlamı" (context) sunar.

**4. Veri İzolasyonu ve Ölçeklenebilirlik:**
[cite_start]Geliştirdiğimiz son sınıf yapısı, her kullanıcı veya her makale için ayrı koleksiyonlar oluşturma yeteneğine sahiptir. [cite: 120] Bu, kullanıcı verilerinin birbirine karışmasını engeller (veri izolasyonu) ve gelecekte binlerce makaleyi veya kullanıcıyı yönetecek şekilde sistemin kolayca ölçeklenmesine olanak tanır.

Kısacası, **VectorStore modülü, ham ve anlamsız metin yığınları ile cevap üreten yapay zeka arasında bir köprü görevi görür.** Bu köprü olmadan, projenin temel hedefi olan akademik makalelerle "sohbet etme" yeteneği mümkün olmazdı.